In [4]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [ ]:
model = tf.keras.models.load_model("final_model.h5")

In [2]:
path = "haarcascade_frontalface_default.xml"
fontScale = 1.5
font = cv2.FONT_HERSHEY_PLAIN

In [5]:
rectangleBGR = (255, 255, 255)
img = np.zeros((500, 500))
text = "Some text in a box!"

(textWidth, textHeight) = cv2.getTextSize(text, font, fontScale=fontScale, thickness=1)[0]

textOffestX = 10
textOffsetY = img.shape[0] - 25

boxCoords = ((textOffestX, textOffestX), (textOffestX + textWidth + 2, textOffsetY - textHeight -2))
cv2.rectangle(img, boxCoords[0], boxCoords[1], rectangleBGR, cv2.FILLED)
cv2.putText(img, text, (textOffestX, textOffsetY), font, fontScale=fontScale, color=(0, 0, 0), thickness=1)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [6]:
cap = cv2.VideoCapture(1)

if not cap.isOpened():
    cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError("Cannot open webcam")

In [ ]:
def detectFace():
    faceCascade = cv2.CascadeClassifier(cv2.samples.findFile('haarcascade_frontalface_default.xml'))
    faces = faceCascade.detectMultiScale(gray, 1.1, 4)
    for x, y, w, h in faces:
        roi_gray = gray[y:y+h, x:x+h]
        roi_color = frame[y:y+h, x:x+h]
        cv2.rectangle(frame, (x, y) , (w+x, y+h), (0, 255, 0), 2)
        allFaces = faceCascade.detectMultiScale(roi_gray)
        if len(allFaces) == 0:
            print('no faces detected')
        else:
            for (ex, ey, ew, eh) in allFaces:
                face_roi = roi_color[ey:ey+eh, ex:ex+eh] # cropping the face
    
    finalImg = cv2.resize(face_roi, (img_size, img_size))
    finalImg = np.expand_dims(finalImg, axis= 0) # adding fourth dimension
    finalImg = finalImg / 255.0   

    return finalImg

In [ ]:
def predict(img):
    model = tf.keras.models.load_model('model.h5')
    pred = model.predict(img)
    return pred

In [ ]:
def predictText(frame, status, font):
    x, y, w, h = 0, 0, 175, 75
    cv2.rectangle(frame, (x1, x1), (x1+w1, y1+h1), (0,0,0), -1)
    cv2.putText(frame, status, (x1 + int(w1/10), y1 + int(h1/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
    cv2.putText(frame, status, (100, 150), font, 3, (0,0,255),2,cv2.LINE_4)
    cv2.rectangle(frame, (x, y), (x+w, y+h), (0,0,255))

In [ ]:
def capture():
    while True:
        ret, frame = cap.read()
        
        detectedFace = detectFace()
        prediction = predict(detectedFace)

        fontScale = 1.5
        font = cv2.FONT_HERSHEY_PLAIN

        if (np.argmax(predictions) == 0):
            status="Angry"
            predictText(frame, status, font)
        elif (np.argmax(predictions) == 1):
            status="Disgust"
            predictText(frame, status, font)
        elif (np.argmax(predictions) == 2):
            status="Fear"
            predictText(frame, status, font)
        elif (np.argmax(predictions) == 3):
            status="Happy"
            predictText(frame, status, font)
        elif (np.argmax(predictions) == 4):
            status="Sad"
            predictText(frame, status, font)
        elif (np.argmax(predictions) == 5):
            status="Surprise"
            predictText(frame, status, font)
        elif (np.argmax(predictions) == 6):
            status="Neutral"
            predictText(frame, status, font)

        cv2.imshow('Face Emotion Recognition',frame)

        if cv2.waitKey(2) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
capture()